In [1]:
import pandas as pd

In [2]:
DC_crime = pd.read_csv('Resources/dc-crimes-search-results.csv')
DC_crime.columns

Index(['NEIGHBORHOOD_CLUSTER', 'CENSUS_TRACT', 'offensegroup', 'LONGITUDE',
       'END_DATE', 'offense-text', 'YBLOCK', 'SHIFT', 'DISTRICT', 'WARD',
       'YEAR', 'offensekey', 'BID', 'sector', 'ucr-rank', 'PSA', 'BLOCK_GROUP',
       'VOTING_PRECINCT', 'XBLOCK', 'BLOCK', 'START_DATE', 'CCN', 'OFFENSE',
       'OCTO_RECORD_ID', 'ANC', 'REPORT_DAT', 'METHOD', 'location',
       'LATITUDE'],
      dtype='object')

In [3]:
DC_crime_data = pd.DataFrame(DC_crime)
DC_crime_data.pop('NEIGHBORHOOD_CLUSTER')
DC_crime_data.pop('CENSUS_TRACT')
DC_crime_data.pop('LONGITUDE')
DC_crime_data.pop('YBLOCK')
DC_crime_data.pop('SHIFT')
DC_crime_data.pop('DISTRICT')
DC_crime_data.pop('WARD')
DC_crime_data.pop('VOTING_PRECINCT')
DC_crime_data.pop('XBLOCK')
DC_crime_data.pop('BLOCK')
DC_crime_data.pop('CCN')
DC_crime_data.pop('OCTO_RECORD_ID')
DC_crime_data.pop('ANC')
DC_crime_data.pop('REPORT_DAT')
DC_crime_data.pop('METHOD')
DC_crime_data.pop('location')
DC_crime_data.pop('LATITUDE')

DC_crime_data.pop('BID')
DC_crime_data.pop('sector')
DC_crime_data.pop('ucr-rank')
DC_crime_data.pop('PSA')
DC_crime_data.pop('BLOCK_GROUP')

DC_crime_data = DC_crime_data[['YEAR', 'START_DATE', 'END_DATE', 'offensegroup', 'OFFENSE', 'offense-text', 'offensekey']]
pd.DataFrame(DC_crime_data['START_DATE'].to_list(), columns=['Start crime Date', 'Start crime time'])
# pd.DataFrame(df["langs"].to_list(), columns=['prim_lang', 'sec_lang'])
# DC_crime_data.rename(columns = {'0':'Date of Crime','1':'Start Time of Crime'}, inplace = True)
DC_crime_data.head()


ValueError: Shape of passed values is (56034, 1), indices imply (56034, 2)

In [4]:
DC_crime_data.groupby(DC_crime_data['offensekey']).count()

,YEAR,START_DATE,END_DATE,offensegroup,OFFENSE,offense-text
offensekey,,,,,,
property|arson,11,11,10,11,11,11
property|burglary,2240,2240,1962,2240,2240,2240
property|motor vehicle theft,7490,7490,6659,7490,7490,7490
property|theft f/auto,16459,16457,15877,16459,16459,16459
property|theft/other,21900,21896,20201,21900,21900,21900
violent|assault w/dangerous weapon,3040,3039,2371,3040,3040,3040
violent|homicide,420,418,304,420,420,420
violent|robbery,4138,4137,3406,4138,4138,4138
violent|sex abuse,336,336,310,336,336,336
